In [3]:
import sys
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [44]:
#hyperparameters 

IMG_SIZE = 224
BATCH_SIZE = 64
CLASSES = 1
EPOCH = 100

IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LR = 0.001

In [5]:

train_csv_path = '/data/Pathology/SPIE/training_set/breastpathq/datasets/train_labels.csv'
test_csv_path = '/data/Pathology/SPIE/training_set/breastpathq/datasets/val_labels.csv'
train_csv = pd.read_csv(train_csv_path)
test_csv = pd.read_csv(test_csv_path)
train_test_dict = {'train':train_csv, 'test': test_csv}
len(train_test_dict['test'])

185

In [7]:
train_csv.head()

,slide,rid,y
0,99861,1,0.40
1,99861,2,0.40
2,99861,3,0.15
3,99861,4,0.10
4,99861,5,0.07


In [6]:
train_transformer = transforms.Compose([transforms.Resize((256,256)),
                                 transforms.RandomRotation((-180, 180)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomResizedCrop((224,224)),  
                                 transforms.ToTensor(),
                                       transforms.Normalize(IMG_MEAN,IMG_STD)]
                                )
test_transformer = transforms.Compose([transforms.Resize((256,256)),
                                  transforms.CenterCrop((224,224)),
                                 transforms.ToTensor(),
                                      transforms.Normalize(IMG_MEAN,IMG_STD)])
train_test_transformer = {'train':train_transformer, 'test':test_transformer}

In [8]:
train_dir = '/data/Pathology/SPIE/training_set/breastpathq/datasets/train/'
val_dir = '/data/Pathology/SPIE/training_set/breastpathq/datasets/validation/'

class SPIE(Dataset):
    def __init__(self, phase, transforms=True):
        self.phase = phase
        self.transforms = transforms
        if phase == 'train':
            self.df = train_test_dict['train']
            self.dir =  '/data/Pathology/SPIE/training_set/breastpathq/datasets/train/'
            self.transformer = train_test_transformer['train']
        else:
            self.df = train_test_dict['test']
            self.dir = '/data/Pathology/SPIE/training_set/breastpathq/datasets/validation/'
            self.transformer = train_test_transformer['test']
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        slide = self.df.loc[idx,'slide']
        rid = self.df.loc[idx, 'rid']
        img_path = self.dir + str(slide)+'_'+str(rid)+'.tif'
        img = Image.open(img_path).convert('RGB')
        label = self.df.loc[idx, 'y']
        
        if self.transforms:
            img = self.transformer(img)
        
        return (img, label)

In [45]:
train_dataset = SPIE('train')
test_dataset = SPIE('test')

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [43]:
from torchvision.models import densenet169
model = densenet169(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

fc_features = model.classifier.in_features
model.classifier = torch.nn.Linear(fc_features, CLASSES)
model = torch.nn.Sequential(model, torch.nn.Sigmoid())
model.to(DEVICE)

# class Model()

Sequential(
  (0): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, tra

0.features.conv0.weight
0.features.norm0.weight
0.features.norm0.bias
0.features.denseblock1.denselayer1.norm1.weight
0.features.denseblock1.denselayer1.norm1.bias
0.features.denseblock1.denselayer1.conv1.weight
0.features.denseblock1.denselayer1.norm2.weight
0.features.denseblock1.denselayer1.norm2.bias
0.features.denseblock1.denselayer1.conv2.weight
0.features.denseblock1.denselayer2.norm1.weight
0.features.denseblock1.denselayer2.norm1.bias
0.features.denseblock1.denselayer2.conv1.weight
0.features.denseblock1.denselayer2.norm2.weight
0.features.denseblock1.denselayer2.norm2.bias
0.features.denseblock1.denselayer2.conv2.weight
0.features.denseblock1.denselayer3.norm1.weight
0.features.denseblock1.denselayer3.norm1.bias
0.features.denseblock1.denselayer3.conv1.weight
0.features.denseblock1.denselayer3.norm2.weight
0.features.denseblock1.denselayer3.norm2.bias
0.features.denseblock1.denselayer3.conv2.weight
0.features.denseblock1.denselayer4.norm1.weight
0.features.denseblock1.densela

In [41]:
optimizer = optim.SGD([{'params':model[0].classifier.parameters(), 'lr':0.0004}])
#                        {'params':model.layer4.parameters(),'lr':0.00001}
criterion = nn.MSELoss()

In [37]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.float().to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    print ('Train Epoch: {}\t Loss: {:.6f}\n'.format(epoch,loss.item()))

    
# def test(model, device, test_loader, optimizer):
#     model.eval()
#     test_loss = 0
#     correct = 0
# #     pred_result = []
#     with torch.no_grad():
#         for i,data in enumerate(test_loader):          
#             x,y= data
#             x=x.to(device)
#             y=y.float().to(device)
#             optimizer.zero_grad()
#             y_hat = model(x)
#             test_loss = criterion(y_hat, y).item() # sum up batch loss
#             pred = y_hat.max(1, keepdim=True)[1] # get the index of the max log-probability
            
#             correct += pred.eq(y.view_as(pred)).sum().item()
# #     test_loss /= len(test_loader.dataset)
#     acc = 100. * correct / len(train_test_dict['test'])
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(train_test_dict['test']),
#         acc))
    
# #     print('-----------------------')
#     return acc

#!/usr/bin/env python
def error_analysis(model, test_loader, DEVICE=DEVICE):
    y_pred =[]
    y_true = []
    for i,data in enumerate(test_loader):
        x, y = data
        y = np.array(y.cpu())
        y_true.append(y)

        x = x.to(DEVICE)
        y_hat = model(x)
        pred = y_hat.max(1, keepdim=True)[1]
        pred = np.array(pred.cpu())
        y_pred.append(pred)
    
    y_pred_list = []
    y_true_list = []
    for i in range(len(y_pred)):
        y_pred_list += list(y_pred[i].ravel())
    for i in range(len(y_true)):
        y_true_list += list(y_true[i].ravel())
    return np.array(y_true_list), np.array(y_pred_list)

def predprob(x, y, initial_lexsort=True):
    """
    Calculates the prediction probability. Adapted from scipy's implementation of Kendall's Tau

    Note: x should be the truth labels.

    Parameters
    ----------
    x, y : array_like
        Arrays of rankings, of the same shape. If arrays are not 1-D, they will
        be flattened to 1-D.
    initial_lexsort : bool, optional
        Whether to use lexsort or quicksort as the sorting method for the
        initial sort of the inputs. Default is lexsort (True), for which
        `predprob` is of complexity O(n log(n)). If False, the complexity is
        O(n^2), but with a smaller pre-factor (so quicksort may be faster for
        small arrays).
    Returns
    -------
    Prediction probability : float

    Notes
    -----
    The definition of prediction probability that is used is:
      p_k = (((P - Q) / (P + Q + T)) + 1)/2
    where P is the number of concordant pairs, Q the number of discordant
    pairs, and T the number of ties only in `y`.
    References
    ----------
    Smith W.D, Dutton R.C, Smith N.T. (1996) A measure of association for assessing prediction accuracy
    that is a generalization of non-parametric ROC area. Stat Med. Jun 15;15(11):1199-215
    """

    x = np.asarray(x).ravel()
    y = np.asarray(y).ravel()

    if not x.size or not y.size:
        return (np.nan, np.nan)  # Return NaN if arrays are empty

    n = np.int64(len(x))
    temp = list(range(n))  # support structure used by mergesort
    # this closure recursively sorts sections of perm[] by comparing
    # elements of y[perm[]] using temp[] as support
    # returns the number of swaps required by an equivalent bubble sort

    def mergesort(offs, length):
        exchcnt = 0
        if length == 1:
            return 0
        if length == 2:
            if y[perm[offs]] <= y[perm[offs+1]]:
                return 0
            t = perm[offs]
            perm[offs] = perm[offs+1]
            perm[offs+1] = t
            return 1
        length0 = length // 2
        length1 = length - length0
        middle = offs + length0
        exchcnt += mergesort(offs, length0)
        exchcnt += mergesort(middle, length1)
        if y[perm[middle - 1]] < y[perm[middle]]:
            return exchcnt
        # merging
        i = j = k = 0
        while j < length0 or k < length1:
            if k >= length1 or (j < length0 and y[perm[offs + j]] <=
                                                y[perm[middle + k]]):
                temp[i] = perm[offs + j]
                d = i - j
                j += 1
            else:
                temp[i] = perm[middle + k]
                d = (offs + i) - (middle + k)
                k += 1
            if d > 0:
                exchcnt += d
            i += 1
        perm[offs:offs+length] = temp[0:length]
        return exchcnt

    # initial sort on values of x and, if tied, on values of y
    if initial_lexsort:
        # sort implemented as mergesort, worst case: O(n log(n))
        perm = np.lexsort((y, x))
    else:
        # sort implemented as quicksort, 30% faster but with worst case: O(n^2)
        perm = list(range(n))
        perm.sort(key=lambda a: (x[a], y[a]))

    # compute joint ties
    first = 0
    t = 0
    for i in xrange(1, n):
        if x[perm[first]] != x[perm[i]] or y[perm[first]] != y[perm[i]]:
            t += ((i - first) * (i - first - 1)) // 2
            first = i
    t += ((n - first) * (n - first - 1)) // 2

    # compute ties in x
    first = 0
    u = 0
    for i in xrange(1,n):
        if x[perm[first]] != x[perm[i]]:
            u += ((i - first) * (i - first - 1)) // 2
            first = i
    u += ((n - first) * (n - first - 1)) // 2

    # count exchanges
    exchanges = mergesort(0, n)
    # compute ties in y after mergesort with counting
    first = 0
    v = 0
    for i in xrange(1,n):
        if y[perm[first]] != y[perm[i]]:
            v += ((i - first) * (i - first - 1)) // 2
            first = i
    v += ((n - first) * (n - first - 1)) // 2

    tot = (n * (n - 1)) // 2
    if tot == u or tot == v:
        return (np.nan, np.nan)    # Special case for all ties in both ranks

    p_k = (((tot - (v + u - t)) - 2.0 * exchanges) / (tot - u) + 1)/2

    return p_k


def test(model, test_loader, DEVICE=DEVICE):
    
    x, y = error_analysis(model, test_loader, DEVICE=DEVICE)
    p_k = predprob(x, y)
    print('current p_k value: ', p_k, '\n')
#     print('------------------------------')
    return p_k

In [46]:
max_p_k = 0
max_epoch = 0

save_path = '/data/yaoms/model/densenet-169_0723.pth'
for epoch in range(EPOCH):
    train(model, DEVICE, train_loader, optimizer, epoch)
    p_k = test(model, test_loader, DEVICE)
    if p_k >= max_p_k:
        max_p_k = p_k
        max_epoch = epoch
        if os.path.exists(save_path):
            os.remove(save_path)
        torch.save(model, save_path)
    print('current best p_k: ', max_p_k)
    print('-----------------------')
print('Result: at {} epoch, achieved best acc: {:.0f}'.format(max_epoch, max_p_k))

Train Epoch: 0	 Loss: 0.109785

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 1	 Loss: 0.143600

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 2	 Loss: 0.101962

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 3	 Loss: 0.095097

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 4	 Loss: 0.116265

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 5	 Loss: 0.111675

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 6	 Loss: 0.129345

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 7	 Loss: 0.085280

('current p_k value: ', (nan, nan), '

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 62	 Loss: 0.140902

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 63	 Loss: 0.114357

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 64	 Loss: 0.089655

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 65	 Loss: 0.115903

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 66	 Loss: 0.132157

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 67	 Loss: 0.093256

('current p_k value: ', (nan, nan), '\n')
('current best p_k: ', (nan, nan))
-----------------------
Train Epoch: 68	 Loss: 0.106426

('current p_k value: ', (nan, nan), '\n')
('current best p_k: 

ValueError: Unknown format code 'f' for object of type 'str'